## 초기 세팅 및 데이터 준비

In [119]:
import os

# OpenAI API를 활용 시
# === 필수: OPENAI_API_KEY ===
os.environ["OPENAI_API_KEY"] = "S13P32S101-f2a0e414-d636-44a5-982a-8230a5edb0db"

# === 선택: 팀 전용 게이트웨이 ===
# 예: "https://your-team-endpoint.example.com/v1"
os.environ["OPENAI_BASE_URL"] = "https://gms.ssafy.io/gmsapi/api.openai.com/v1"

os.environ.setdefault("OPENAI_MODEL", "gpt-4o")

BASE_MODEL_ID = os.getenv("BASE_MODEL_ID", "Qwen/Qwen3-VL-4B-Instruct")


In [120]:

import json, csv, random, time
from typing import List, Dict, Any

import pandas as pd
from tqdm import tqdm
from tenacity import retry, stop_after_attempt, wait_exponential

# OpenAI SDK (신/구버전 호환)
from openai import OpenAI

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "")
OPENAI_BASE_URL = os.getenv("OPENAI_BASE_URL", "https://api.openai.com/v1")
OPENAI_MODEL = os.getenv("OPENAI_MODEL", "gpt-4.1")
BASE_MODEL_ID = os.getenv("BASE_MODEL_ID", "Qwen/Qwen3-VL-4B-Instruct")
random.seed(7)

print("OPENAI_API_KEY", OPENAI_API_KEY)
print("OPENAI_BASE_URL", OPENAI_BASE_URL)
print("OPENAI_MODEL:", OPENAI_MODEL)
print("BASE_MODEL_ID:", BASE_MODEL_ID)


OPENAI_API_KEY S13P32S101-f2a0e414-d636-44a5-982a-8230a5edb0db
OPENAI_BASE_URL https://gms.ssafy.io/gmsapi/api.openai.com/v1
OPENAI_MODEL: gpt-4o
BASE_MODEL_ID: Qwen/Qwen3-VL-4B-Instruct


In [93]:
import pandas as pd
from pathlib import Path
import re

base_dir = Path("test")

# 모든 txt 파일 경로 불러오기 (재귀적으로)
txt_files = list(base_dir.rglob("*.txt"))

sentences = []

for file in txt_files:
    with open(file, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if line:  # 빈 줄 제외
                # "A." 또는 "B." 제거 (앞에 공백이 있어도 제거)
                line = re.sub(r'^[AB]\.\s*', '', line)

                sentences.append({
                    "file_path": str(file),
                    "sentence": line.strip("A.")
                })

df = pd.DataFrame(sentences)

display(df.tail(20))
print(f"총 문장 수: {len(df)}")


,file_path,sentence
972,test/TS_1.shopping/01.AS문의/shopping1_0631.txt,팔분에서 넘어가지 않아요
973,test/TS_1.shopping/01.AS문의/shopping1_0631.txt,빈 통으로 해주셨을 때는 통이 돌아는 가나요
974,test/TS_1.shopping/01.AS문의/shopping1_0631.txt,돌아는 가는데 탈수하면은 눈에 안 보이게 막 돌아가야 되잖아요
975,test/TS_1.shopping/01.AS문의/shopping1_0631.txt,근데 무슨 소리가 이렇게 털커덕 소리나면서 천천히 돌아요
976,test/TS_1.shopping/01.AS문의/shopping1_0631.txt,그러세요 알겠습니다 많이 불편하셨겠습니다
977,test/TS_1.shopping/01.AS문의/shopping1_0631.txt,금방 접수 진행해 드릴게요 잠시만요
978,test/TS_1.shopping/01.AS문의/.ipynb_checkpoints/...,네 많이 걱정하셨을 텐데요
979,test/TS_1.shopping/01.AS문의/.ipynb_checkpoints/...,일단 기사가 다시 가서 확인을 좀 드릴거고요
980,test/TS_1.shopping/01.AS문의/.ipynb_checkpoints/...,일정 확인해 볼텐데 혹시 방문 시간대는 언제가 좀 편하시겠어요 고객님
981,test/TS_1.shopping/01.AS문의/.ipynb_checkpoints/...,지금 이 지금 시간대가 좋은데요


총 문장 수: 992


In [95]:
# 데이터 후처리
# 원래 라벨링 전에 해야하는데, 실수로 먼저 돌려서 토큰 아끼기 위해 후처리

# 중복된 행 제거 (완전히 같은 행 기준)
df = df.drop_duplicates()

# 만약 특정 컬럼 기준으로만 중복 제거하고 싶다면
df = df.drop_duplicates(subset=['file_path', 'sentence'])

# 인덱스 리셋 (선택사항)
df = df.reset_index(drop=True)

# .ipynb_checkpoints 가 포함된 행 제거
df = df[~df['file_path'].str.contains('/.ipynb', na=False)]

# 인덱스 리셋 (선택)
df = df.reset_index(drop=True)

display(df)

,file_path,sentence
0,test/TS_1.shopping/01.AS문의/shopping1_0625.txt,네 반갑습니다 상담사 #@이름#입니다
1,test/TS_1.shopping/01.AS문의/shopping1_0625.txt,네 아까 제가 지금 상담을 하는 건데 들으세요
2,test/TS_1.shopping/01.AS문의/shopping1_0625.txt,#@소속#에서 판매를 어떤 물건을 하든지 하게 되면은 판매 정책과 사업부가 있죠
3,test/TS_1.shopping/01.AS문의/shopping1_0625.txt,상담하는 거는 그 사업을 위해서 상담하는 거고요
4,test/TS_1.shopping/01.AS문의/shopping1_0625.txt,그니까 저희는 유지관리를 상담해 드리는 상담실입니다
...,...,...
972,test/TS_1.shopping/01.AS문의/shopping1_0631.txt,빈 통으로 해주셨을 때는 통이 돌아는 가나요
973,test/TS_1.shopping/01.AS문의/shopping1_0631.txt,돌아는 가는데 탈수하면은 눈에 안 보이게 막 돌아가야 되잖아요
974,test/TS_1.shopping/01.AS문의/shopping1_0631.txt,근데 무슨 소리가 이렇게 털커덕 소리나면서 천천히 돌아요
975,test/TS_1.shopping/01.AS문의/shopping1_0631.txt,그러세요 알겠습니다 많이 불편하셨겠습니다


## 모델 로드 및 라벨링 로직

In [118]:
from typing import Optional
import torch
from transformers import pipeline

def safe_parse_json(payload: Any) -> Dict[str, Any]:
    """
    Robust JSON extractor:
    - payload가 dict/list/str 어떤 형태여도 처리
    - assistant.content만 추출
    - 바깥 { ... } 슬라이스
    - 흔한 깨짐 복구: 개행/탭/작은따옴표/트레일링 콤마
    """
    # 1) assistant 텍스트만 뽑기
    text = ""

    # 케이스 A: {'role':'assistant','content':'{...}'}
    if isinstance(payload, dict) and "content" in payload:
        text = str(payload.get("content", ""))

    # 케이스 B: [{'generated_text': '...'}] 또는 [{'generated_text': [{'role':..., 'content': '...'}]}]
    elif isinstance(payload, list) and payload:
        first = payload[0]
        gen = first.get("generated_text") if isinstance(first, dict) else None
        if isinstance(gen, str):
            text = gen
        elif isinstance(gen, list):
            # role 리스트라면 뒤에서부터 assistant content 찾기
            for msg in reversed(gen):
                if isinstance(msg, dict) and msg.get("role") == "assistant":
                    text = msg.get("content", "")
                    break
            if not text:
                for msg in gen:
                    if isinstance(msg, dict) and "content" in msg:
                        text = msg["content"]
                        break
        else:
            # list인데 포맷이 다르면 문자열로 캐스팅
            text = str(first)

    # 케이스 C: 이미 문자열
    elif isinstance(payload, str):
        text = payload
    else:
        text = str(payload or "")

    s = text.strip()
    if not s:
        return {}

    # 2) 바깥쪽 { ... }만 슬라이스
    if "{" in s and "}" in s:
        s = s[s.find("{"): s.rfind("}") + 1]

    # 3) 1차 파싱 시도
    try:
        return json.loads(s)
    except Exception:
        pass

    # 4) 복구: 개행/탭 제거, 작은따옴표 -> 큰따옴표
    s_fix = s.replace("\n", " ").replace("\t", " ")
    # JSON 내 한국어/문장부호엔 영향 적으니 전체 치환 허용
    s_fix = s_fix.replace("'", '"')

    # 5) 복구: 트레일링 콤마 제거 (", }", ", ]")
    s_fix = re.sub(r",\s*([}\]])", r"\1", s_fix)

    # 6) 최종 파싱 시도
    try:
        return json.loads(s_fix)
    except Exception:
        return {}


### 프롬프트 및 few shot 예시

In [ ]:
# ✅ 분류 기준 (객체 루트로 통일)
SYSTEM_PROMPT = """너는 '일정 관련 문장(1) / 비일정(0)' 분류기다.
반드시 아래 JSON만 출력해.
필드:
- label: 0 또는 1
- reason: 한 줄 한국어 이유
- certainty: 0~100 정수 (확신도)
- time_cue: 시간/날짜 단서 포함 여부 (true/false)
- schedule_cue: 예약/방문/배정/가능 등 일정 단서 포함 여부 (true/false)
출력 예: {"label":1,"reason":"내일 11시 방문 요청","certainty":92,"time_cue":true,"schedule_cue":true}
규칙: JSON 외 불필요한 텍스트 금지.
기준: 시간/날짜/장소/예약 생성·변경·확인 요청은 1, 단순 안내/사과/인사는 0.
애매하면 단서 없으면 0, 단서 2개↑면 1.
"""


FEW_SHOTS = [
    # 기존 두 개
    {"role":"user","content":"문장: '열한 시 삼십 분에 기사 연락 드리고 방문 드릴게요'"},
    {"role":"assistant","content":'{"items":[{"sentence":"열한 시 삼십 분에 기사 연락 드리고 방문 드릴게요","label":1}]}'},
    {"role":"user","content":"문장: '같은 부분이면 비용이 안 나옵니다'"},
    {"role":"assistant","content":'{"items":[{"sentence":"같은 부분이면 비용이 안 나옵니다","label":0}]}'},

    # 추가 few-shot들 (객체 루트로 통일)
    {"role":"user","content":"문장: '내일 2시에 콘티 리뷰 시간 괜찮으세요?'"},
    {"role":"assistant","content":'{"items":[{"sentence":"내일 2시에 콘티 리뷰 시간 괜찮으세요?","label":1}]}'},
    {"role":"user","content":"문장: '네 지금 바로 갈 거예요'"},
    {"role":"assistant","content":'{"items":[{"sentence":"네 지금 바로 갈 거예요","label":0}]}'},
    {"role":"user","content":"문장: '오늘 잉킹 세션 취소합니다'"},
    {"role":"assistant","content":'{"items":[{"sentence":"오늘 잉킹 세션 취소합니다","label":1}]}'},
    {"role":"user","content":"문장: '전화번호 어떻게 되세요'"},
    {"role":"assistant","content":'{"items":[{"sentence":"전화번호 어떻게 되세요","label":0}]}'},
    {"role":"user","content":"문장: 'PSD 업로드 완료했습니다'"},
    {"role":"assistant","content":'{"items":[{"sentence":"PSD 업로드 완료했습니다","label":1}]}'},
    {"role":"user","content":"문장: '고객님 죄송하지만 예약은 한 시간 전부터 가능하십니다'"},
    {"role":"assistant","content":'{"items":[{"sentence":"고객님 죄송하지만 예약은 한 시간 전부터 가능하십니다","label":0}]}'},
    {"role":"user","content":"문장: '3시쯤 한 번 보시죠? 길게는 말고요'"},
    {"role":"assistant","content":'{"items":[{"sentence":"3시쯤 한 번 보시죠? 길게는 말고요","label":1}]}'},
]


In [121]:
# ==== OpenAI 클라이언트 (한 번만 생성) ====
OPENAI_BASE_URL = os.getenv("OPENAI_BASE_URL")  # 커스텀 게이트웨이 쓰면 설정, 아니면 None
client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    base_url=OPENAI_BASE_URL if OPENAI_BASE_URL else None
)

# ==== 오픈소스 모델 로드 (한 번만 생성) ====
pipe = None  # 전역 변수
def get_LLM():
    global pipe
    if pipe is None:
        if not BASE_MODEL_ID:
            raise RuntimeError("BASE_MODEL_ID not set.")
        pipe = pipeline("text-generation", model=BASE_MODEL_ID)
    return pipe

@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=2, max=10))
def begin_pipeline(system_prompt: str, user_prompt: str) -> str:
    pipe = get_LLM()
    if pipe:
        try:
            messages = [
                {
                    "role": "system",
                    "content": system_prompt
                },
                {
                    "role": "user",
                    "content": user_prompt
                }
            ]
            outputs = pipe(messages, max_new_tokens=2048)
            return outputs[0].get('generated_text')[2]["content"]
        except Exception:
            pass


In [125]:
resp = begin_pipeline(SYSTEM_PROMPT, "돌아는 가는데 탈수하면은 눈에 안 보이게 막 돌아가야 되잖아요")
print(resp)

{"label":0,"reason":"일정 관련 단서 없음","certainty":90,"time_cue":false,"schedule_cue":false}


In [122]:
# pip install openai pandas tqdm tenacity
import os, json, re, statistics
import pandas as pd
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type
from openai import OpenAI
from collections import Counter

def _safe_parse_json(text: str):
    # 코드블록/잡텍스트 섞여도 {} 첫/끝만 추출
    m = re.search(r'\{.*\}', text, re.S)
    if not m: 
        return None
    try:
        return json.loads(m.group(0))
    except:
        return None

def classify_n(sentence: str, model="gpt-4o-mini", n=5, temperature=0.3, LLM=True):
    # 1) n 지원이면 한 번에
    if LLM:
        resp = begin_pipeline(SYSTEM_PROMPT, sentence)
    else:
        resp = client.chat.completions.create(
            model=model,
            temperature=temperature,
            n=n,
            messages=[
                {"role":"system","content":SYSTEM_PROMPT},
                {"role":"user","content":sentence}
            ]
        )
    outs = []
    for ch in resp.choices:
        data = _safe_parse_json(ch.message.content or "")
        if data and "label" in data:
            outs.append({
                "label": int(data.get("label", 0)),
                "certainty": int(data.get("certainty", 50)),
                "time_cue": bool(data.get("time_cue", False)),
                "schedule_cue": bool(data.get("schedule_cue", False)),
                "reason": data.get("reason","")
            })
    return outs

def vote_result(outs, conservative_default=0):
    # 다수결 + 동률시 확신도 평균 높은 쪽
    if not outs:
        return {"label": conservative_default, "agree_rate": 0.0, "certainty_avg": 0, "outs": []}
    labels = [o["label"] for o in outs]
    cnt = Counter(labels)
    top_label, top_count = cnt.most_common(1)[0]
    agree_rate = top_count / len(outs)
    # 동률 처리
    if len([1 for v in cnt.values() if v == top_count]) > 1:
        # 확신도 평균으로 승부
        cert_avg = {
            lab: statistics.mean([o["certainty"] for o in outs if o["label"]==lab]) 
            for lab in cnt.keys()
        }
        # 같으면 보수적으로 0
        if cert_avg.get(1,0) == cert_avg.get(0,0):
            top_label = conservative_default
        else:
            top_label = max(cert_avg.items(), key=lambda x: x[1])[0]
    certainty_avg = int(statistics.mean([o["certainty"] for o in outs if o["label"]==top_label]))
    return {
        "label": int(top_label),
        "agree_rate": round(agree_rate, 3),
        "certainty_avg": certainty_avg,
        "outs": outs
    }

In [126]:
from tqdm import tqdm
tqdm.pandas(desc="Labeling sentences")

def label_with_vote(sentence: str, n=5):
    outs = classify_n(sentence, n=n, temperature=0.3, LLM=False)
    voted = vote_result(outs)
    voted["accept"] = (voted["agree_rate"] >= 0.8 and voted["certainty_avg"] >= 70)
    return voted
    
def run_labeling(df: pd.DataFrame, text_col="sentence", n=5):
    results = df[text_col].progress_apply(lambda s: label_with_vote(str(s), n=n))
    df["label_pred"] = results.apply(lambda r: r["label"])
    df["agree_rate"] = results.apply(lambda r: r["agree_rate"])
    df["certainty_avg"] = results.apply(lambda r: r["certainty_avg"])
    df["accept"] = results.apply(lambda r: r["accept"])
    return df

In [127]:
df_labeled = run_labeling(df)

Labeling sentences: 100%|██████████| 977/977 [26:32<00:00,  1.63s/it]


In [128]:
# # df는 이미 sentence 컬럼을 포함한 상태라고 가정
# df_labeled = label_with_llm(df, text_col="sentence", batch_size=20, model=OPENAI_MODEL)

# 저장 및 확인
df_labeled.to_csv("labeled_sentences_voted.csv", index=False)
df_labeled.head(10)

,file_path,sentence,label_pred,agree_rate,certainty_avg,accept
0,test/TS_1.shopping/01.AS문의/shopping1_0625.txt,네 반갑습니다 상담사 #@이름#입니다,0,1.0,95,True
1,test/TS_1.shopping/01.AS문의/shopping1_0625.txt,네 아까 제가 지금 상담을 하는 건데 들으세요,0,1.0,84,True
2,test/TS_1.shopping/01.AS문의/shopping1_0625.txt,#@소속#에서 판매를 어떤 물건을 하든지 하게 되면은 판매 정책과 사업부가 있죠,0,1.0,87,True
3,test/TS_1.shopping/01.AS문의/shopping1_0625.txt,상담하는 거는 그 사업을 위해서 상담하는 거고요,0,1.0,85,True
4,test/TS_1.shopping/01.AS문의/shopping1_0625.txt,그니까 저희는 유지관리를 상담해 드리는 상담실입니다,0,1.0,85,True
5,test/TS_1.shopping/01.AS문의/shopping1_0625.txt,사업을 진행하는 데에서는 고객님과 통화하시는 부분이 아니고요,0,1.0,85,True
6,test/TS_1.shopping/01.AS문의/shopping1_0625.txt,저희가 내용을 전달해 드리는 겁니다,0,1.0,85,True
7,test/TS_1.shopping/01.AS문의/shopping1_0625.txt,그렇죠 상담하시는 그러니까 그 상담을 그거 밖에 안 되잖아요,0,1.0,85,True
8,test/TS_1.shopping/01.AS문의/shopping1_0625.txt,그래서 그 이외에 총괄적으로 사업을 지휘하는 데가 어딥니까,0,1.0,85,True
9,test/TS_1.shopping/01.AS문의/shopping1_0625.txt,거기의 전화번호 알려달라니까,0,1.0,85,True


In [129]:
# df_labeled = pd.read_csv("labeled_sentences.csv")
display(df_labeled)

,file_path,sentence,label_pred,agree_rate,certainty_avg,accept
0,test/TS_1.shopping/01.AS문의/shopping1_0625.txt,네 반갑습니다 상담사 #@이름#입니다,0,1.0,95,True
1,test/TS_1.shopping/01.AS문의/shopping1_0625.txt,네 아까 제가 지금 상담을 하는 건데 들으세요,0,1.0,84,True
2,test/TS_1.shopping/01.AS문의/shopping1_0625.txt,#@소속#에서 판매를 어떤 물건을 하든지 하게 되면은 판매 정책과 사업부가 있죠,0,1.0,87,True
3,test/TS_1.shopping/01.AS문의/shopping1_0625.txt,상담하는 거는 그 사업을 위해서 상담하는 거고요,0,1.0,85,True
4,test/TS_1.shopping/01.AS문의/shopping1_0625.txt,그니까 저희는 유지관리를 상담해 드리는 상담실입니다,0,1.0,85,True
...,...,...,...,...,...,...
972,test/TS_1.shopping/01.AS문의/shopping1_0631.txt,빈 통으로 해주셨을 때는 통이 돌아는 가나요,0,1.0,85,True
973,test/TS_1.shopping/01.AS문의/shopping1_0631.txt,돌아는 가는데 탈수하면은 눈에 안 보이게 막 돌아가야 되잖아요,0,1.0,85,True
974,test/TS_1.shopping/01.AS문의/shopping1_0631.txt,근데 무슨 소리가 이렇게 털커덕 소리나면서 천천히 돌아요,0,1.0,85,True
975,test/TS_1.shopping/01.AS문의/shopping1_0631.txt,그러세요 알겠습니다 많이 불편하셨겠습니다,0,1.0,85,True


In [130]:
# 데이터 후처리
# 원래 라벨링 전에 해야하는데, 실수로 먼저 돌려서 토큰 아끼기 위해 후처리

# 중복된 행 제거 (완전히 같은 행 기준)
df_labeled = df_labeled.drop_duplicates()

# 만약 특정 컬럼 기준으로만 중복 제거하고 싶다면
df_labeled = df_labeled.drop_duplicates(subset=['file_path', 'sentence'])

# 인덱스 리셋 (선택사항)
df_labeled = df_labeled.reset_index(drop=True)

# .ipynb_checkpoints 가 포함된 행 제거
df_labeled = df_labeled[~df_labeled['file_path'].str.contains('/.ipynb', na=False)]

# 인덱스 리셋 (선택)
df_labeled = df_labeled.reset_index(drop=True)

In [133]:
# 일정 관련 대화 수, 비율 점검
schedule_labeled = df_labeled["label_pred"] == 1

display(df_labeled)
display(df_labeled[schedule_labeled])
print("일정으로 labeling 된 데이터 수 :", len(df_labeled[schedule_labeled]["file_path"]))

,file_path,sentence,label_pred,agree_rate,certainty_avg,accept
0,test/TS_1.shopping/01.AS문의/shopping1_0625.txt,네 반갑습니다 상담사 #@이름#입니다,0,1.0,95,True
1,test/TS_1.shopping/01.AS문의/shopping1_0625.txt,네 아까 제가 지금 상담을 하는 건데 들으세요,0,1.0,84,True
2,test/TS_1.shopping/01.AS문의/shopping1_0625.txt,#@소속#에서 판매를 어떤 물건을 하든지 하게 되면은 판매 정책과 사업부가 있죠,0,1.0,87,True
3,test/TS_1.shopping/01.AS문의/shopping1_0625.txt,상담하는 거는 그 사업을 위해서 상담하는 거고요,0,1.0,85,True
4,test/TS_1.shopping/01.AS문의/shopping1_0625.txt,그니까 저희는 유지관리를 상담해 드리는 상담실입니다,0,1.0,85,True
...,...,...,...,...,...,...
972,test/TS_1.shopping/01.AS문의/shopping1_0631.txt,빈 통으로 해주셨을 때는 통이 돌아는 가나요,0,1.0,85,True
973,test/TS_1.shopping/01.AS문의/shopping1_0631.txt,돌아는 가는데 탈수하면은 눈에 안 보이게 막 돌아가야 되잖아요,0,1.0,85,True
974,test/TS_1.shopping/01.AS문의/shopping1_0631.txt,근데 무슨 소리가 이렇게 털커덕 소리나면서 천천히 돌아요,0,1.0,85,True
975,test/TS_1.shopping/01.AS문의/shopping1_0631.txt,그러세요 알겠습니다 많이 불편하셨겠습니다,0,1.0,85,True


,file_path,sentence,label_pred,agree_rate,certainty_avg,accept
37,test/TS_1.shopping/01.AS문의/shopping1_0639.txt,기사님 출장 예약할게요,1,1.0,85,True
48,test/TS_1.shopping/01.AS문의/shopping1_0639.txt,언제쯤 가능한가요 제일 빠른 시간이,1,1.0,85,True
49,test/TS_1.shopping/01.AS문의/shopping1_0639.txt,예 고객님 강서구 #@주소# 같은 경우에는 오늘도 방문 가능하시고요,1,1.0,85,True
50,test/TS_1.shopping/01.AS문의/shopping1_0639.txt,약속은 열한 시 이십 분이 가장 빠릅니다,1,1.0,85,True
51,test/TS_1.shopping/01.AS문의/shopping1_0639.txt,열한 시 이십 분요 그 이후 시간은요 한 시,1,1.0,85,True
...,...,...,...,...,...,...
902,test/TS_1.shopping/01.AS문의/shopping1_0617.txt,가장 빠른 방문 시간은 오늘 열한 시 삼십 분인데 괜찮으십니까,1,1.0,93,True
904,test/TS_1.shopping/01.AS문의/shopping1_0617.txt,네 그럼 그렇게 예약하겠습니다,1,1.0,85,True
928,test/TS_1.shopping/01.AS문의/shopping1_0647.txt,빨리 도착하면 내일이고 늦어도 모래 정도면 도착할 거예요,1,1.0,85,True
935,test/TS_1.shopping/01.AS문의/shopping1_0619.txt,네 여기서 예약 도와드릴 수 있습니다,1,0.8,85,True


일정으로 labeling 된 데이터 수 : 75
